# 2D Ising model with MC. 
<hr>

### theory
$$ 
H = -J\sum_{<ij>} S_i S_j, S_i = \pm 1
$$
Let J = 1 and h = 0 (No external magnetic field). Ferromagnetic behaviour.


### Packages

In [29]:
import numpy as np
from matplotlib import pyplot as plt
import time

### Useful functions
random spin configuration generating function, Monte-Carlo function, and true free energy calculating function

In [1]:
#make spin configuration with L**2 spins.
def initial(L):
    config = 2*np.random.randint(2, size = (L,L)) - 1
    return config

#Monte-Carlo argorithm with metropolis-hastings. 
def mcmh(config, temp):
    L = int(np.sqrt(config.size))
    for i in range(L**2): #this part is needed to satisfy the definition of one MC sweep.
        x = np.random.randint(0,L)
        y = np.random.randint(0,L)
        s = config[x,y]  
        neigh = config[(x+1)%L, y] + config[x,(y+1)%L] + config[(x-1)%L,y] + config[x,(y-1)%L]
        DE = 2 * s * neigh 

        #accept or reject 
        if DE < 0:
            s *= -1 
        elif np.random.rand() < np.exp(- DE*1/temp):
            s *= -1 
        config[x,y] = s
    return config

## True free energy calculation

In [33]:
# making possible spin configuration using binary number generator
def all_config(L): #
    configs = []
    N = L**2
    for i in range(2**N):
        a = str(bin(i))
        temp = [0] * (N - len(a) + 2)

        for j in range(2, len(a)):
            temp += [int(a[j])]
        configs.append(temp)
    configs = np.array(configs)
    
    all_config = []
    for i in range(2**N):
        all_config.append(np.ones((L,L)) - 2 * configs[i].reshape(L,L)) #이렇게 하면 +1, -1만으로 이루어진 행렬이 나오게 된다. 
      
    return all_config

In [34]:
#true free energy calculation!
def true_free_energy(all_configs, temperature):
    partition = 0
    
    for config in all_configs:
        partition += np.exp(-config / temperature)
    
    return - temperature * np.log(partition)

## Wang-Landau algorithm

In [35]:
#Wang-Landau algorithm
#while looping this algorithm, spin config, g(E) nerver be updated.
def wang_landau(L):
    E = possible_energy_values(L)  
    length = len(E) #L**2-1. L이 짝수일 때는 가능한 에너지의 개수는 L**2-1개.
    log_modi_factor = 1 #처음엔 f = e. 즉, ln_f = 1
    Histogram = np.zeros(length, dtype = float) #가능한 에너지별로 histogram을 만들어준다.
    log_gE = np.zeros(length, dtype = float) #처음엔 g(E)=1. 즉, ln_g(E) = 0.   
    
    config = initial(L) #이 configuration은 loop를 돌리면서 새로 만들지 않는다. 
    E_initial = energy(config) #처음 configuration의 에너지
    
    index_E1, index_E2 = [], [] #에너지를 index로 삼아야 한다. 
    ini_index, cha_index = 0, 0
    
    count_flat = 0 #이 값이 27이 될때까지 loop를 돌려야 한다.
    count_mcsweeps = 0 #1만번 sweep을 돌때마다 histogram이 flat한지 체크한다. 
    
    #making index set
    for j in enumerate(E):
        index_E1.append(j)
    for k in enumerate(E):
        index_E2.append(k)
    
    while count_flat <= 27: 
        count_mcsweeps += 1
        for i in range(L**2): #one MC sweep = randomly pick N(=L**2) spins and change its state(spin flip).
            x = np.random.randint(0,L)
            y = np.random.randint(0,L)  # 0 ~ L-1의 정수를 랜덤하게 하나 고르는 함수. 
            s = config[x,y] #a randomly picked spin
            s *= -1 #우선 spin을 뒤집자.
            
            neigh = config[(x+1)%L, y] + config[x,(y+1)%L] + config[(x-1)%L,y] + config[x,(y-1)%L]
            DE = -2 * s * neigh #spin을 뒤집기 전과 뒤집은 후 에너지 차이. 
            E_changed = E_initial + DE #spin을 뒤집었을 때, 그 configuration의 에너지
                        
            #accept or reject
            ##1. Find index of each energies
            for component1 in index_E1: #index set for reference energy
                if E_initial == component1[1]: 
                    ini_index = component1[0] 

            for component2 in index_E2: #index set for changed energy
                if E_changed == component2[1]:
                    cha_index = component2[0]

            ##2. Calculate probability to change spin configuration
            p = np.exp(log_gE[ini_index] - log_gE[cha_index])

            #np.rand.rand() 0~1 에서 uniformly distributed number를 generate
            #만약, g(E_i) >= g(E_f)라면 ln_p = 0. 따라서 무조건 accept. g(E_f) > g(E_i) 인 경우이더라도 매우 낮은 확률로 accept할 수 있는 조건을 반영
            if p > np.random.rand(): #spin flip is accepted
                s *= 1 #spin flip is accepted. 
                Histogram[cha_index] += 1 #바뀐 애의 histogram에 1을 더함.
                log_gE[cha_index] += log_modi_factor #density of state에 값을 더해줌.
                E_initial = E_changed 

            else: #spin flip is rejected
                s *= -1 #spin flip is rejected. 다시 원상태로 돌림
                Histogram[ini_index] += 1 #initial value의 histogram에 1을 더함
                log_gE[ini_index] += log_modi_factor #density of state에 값을 더해줌. 
                E_initial = E_initial
            
            config[x,y] = s
        #Is histogream flat? 
        #all entries is not less than 80% of the average => Hmin이 average의 80%만 넘으면 된다는 의미겠지?
        if count_mcsweeps % 10000 == 0:
            Havg = np.mean(Histogram)
            Hmax = max(Histogram)
            Hmin = min(Histogram)

            if Havg*0.8 <= Hmin:  
                Histogram = np.zeros(length) #reset histogram
                log_modi_factor = log_modi_factor/2 #update modification factor 
                count_flat += 1 
    return log_gE

In [36]:
def wang_landau_free_energy(log_density_of_state, temperature):
    L = int(np.sqrt(len(log_density_of_state) + 1)) #log_density는 길이가 L**2-1. 즉, 이 값이 가로/세로 spin의 수
    E = possible_energy_values(L)
    log_gE_min = min(log_density_of_state)
    log_density_of_state_new = log_density_of_state - log_gE_min
    
    E_modi = [] #-beta*E term
    for i in range(len(E)):
        E_modi.append(-E[i]/temperature)
    
    exponent = log_density_of_state_new + E_modi #이건 list다
    frac = 0
    
    k = len(E)
    for j in range(k):
        frac += np.exp(exponent[j])
        
    return -temperature * np.log(frac)

#### Subfunctions for Wang-Landau algorithm

In [37]:
#energy calculation function(OK)
def energy(config):
    E = 0
    L = len(config)
    for i in range(L):
        for j in range(L):
            s = config[i,j]
            neigh = config[(i+1)%L, j] + config[i,(j+1)%L] + config[(i-1)%L,j] + config[i,(j-1)%L] 
            E += -neigh * s
            
    return E/2  

#all energy values for even L
def possible_energy_values(L):
    E_min = -2*L**2
    E = []
    for i in range(L**2+1):
        E.append(E_min + 4*i)
    E.pop(1)
    E.pop(-2) 
    return E